# Scrapping Tribunnews 

In [25]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs

In [33]:
data = {
    "keywords": "anies",
    "since_time": "2023-09-01",
    "until_time": "2023-11-10"
}

In [34]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
url = f"https://www.tribunnews.com/index-news?date=2023-9-2&page="
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
cek= soup.find('div', {"class": "paging"})
links= cek.find_all('a')
all_link=[]
for link in links:
        if link and 'href' in link.attrs:
            link_href = link['href']
            all_link.append(link_href)
link=all_link[-1]
parsed_url = urlparse(link)

# Extract the "page" parameter value
page_value = parse_qs(parsed_url.query).get('page', [])[0] if 'page' in parse_qs(parsed_url.query) else None

print("Page value:", page_value)
# body = soup.find('ul', {"class": "lsi"})
# print(body)

Page value: 23


In [35]:
def scrape_links(date, page_number):
    current_date = datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%-m-%-d")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }

    url = f"https://www.tribunnews.com/index-news?date={current_date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.find('ul', {"class": "lsi"})
    articles = body.find_all('li', {"class": "ptb15"})
    
    links = []
    for article in articles:
        header = article.find('h3', {"class": "f16 fbo"})
        if header:
            link = header.find('a')
            if link and 'href' in link.attrs:
                link_href = link['href']
                links.append(link_href)
                
    print(f"Scraped {len(links)} links from page {page_number} url {url}")

    return links


In [37]:
def scrape_link_per_day(date, max_threads=5):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    current_date = datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%-m-%-d")
    url = f"https://www.tribunnews.com/index-news?date={current_date}&page="
    response = requests.get(url + "1", headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    cek = soup.find('div', {"class": "paging"})
    links = cek.find_all('a')
    all_link = []
    for link in links:
        if link and 'href' in link.attrs:
            link_href = link['href']
            all_link.append(link_href)
    last_link = all_link[-1]

    # Extract the "page" parameter value from the last link
    parsed_url = urlparse(last_link)
    page_value = parse_qs(parsed_url.query).get('page', [])[0] if 'page' in parse_qs(parsed_url.query) else None
    print(page_value)
    # Check if page_value is an integer
    try:
        page_number = int(page_value)
    except (TypeError, ValueError):
        page_number = 1

    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        # Use list comprehension to submit tasks to the thread pool
        futures = [executor.submit(scrape_links, current_date, index) for index in range(1, page_number + 1)]

        # Collect results from completed futures
        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [38]:
link=scrape_link_per_day(data["since_time"])

40
Scraped 20 links from page 1 url https://www.tribunnews.com/index-news?date=2023-9-1&page=1
Scraped 20 links from page 2 url https://www.tribunnews.com/index-news?date=2023-9-1&page=2
Scraped 20 links from page 5 url https://www.tribunnews.com/index-news?date=2023-9-1&page=5
Scraped 20 links from page 3 url https://www.tribunnews.com/index-news?date=2023-9-1&page=3
Scraped 20 links from page 4 url https://www.tribunnews.com/index-news?date=2023-9-1&page=4
Scraped 20 links from page 10 url https://www.tribunnews.com/index-news?date=2023-9-1&page=10
Scraped 20 links from page 7 url https://www.tribunnews.com/index-news?date=2023-9-1&page=7
Scraped 20 links from page 9 url https://www.tribunnews.com/index-news?date=2023-9-1&page=9
Scraped 20 links from page 6 url https://www.tribunnews.com/index-news?date=2023-9-1&page=6
Scraped 20 links from page 8 url https://www.tribunnews.com/index-news?date=2023-9-1&page=8
Scraped 20 links from page 12 url https://www.tribunnews.com/index-news?dat

In [39]:
print(len(link))

800


In [ ]:
results=[]

In [ ]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('h1', {"id": "arttitle"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"   
                    # tanggal berita
                    date_elem = soup.find('time', {"class": "grey"})
                    if date_elem:
                        date_text = date_elem.text.strip()
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"class": "side-article txt-article multi-fontsize"})
                    
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                            content_text += p.text.strip() + "\n"
                        
                        if content_text.strip():
                            content_text=content_text
                        else:
                            content_text ="Content not found"
                    else:
                        content_text ="Content not found"

                    return{
                        'title': title_text,
                        'date': date_text,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [ ]:
for url in link:
    data_tribunnews = scrape_url(url)
    print(data["keywords"])
    keywords=data["keywords"]
    if keywords.lower() in data_tribunnews['title'].lower() or keywords.lower() in data_tribunnews['content'].lower():
        print("Data contains keywords:", data_tribunnews['link'])
    else:
        print("News does not contain the specified keywords and will not be inserted into the database. URL:", data_tribunnews['link'])

prabowo
News does not contain the specified keywords and will not be inserted into the database. URL: https://www.tribunnews.com/mata-lokal-memilih/2023/01/01/ganjar-erick-dinilai-mirip-seperti-jokowi-di-2014-yang-muncul-dengan-segudang-prestasi
prabowo
News does not contain the specified keywords and will not be inserted into the database. URL: https://www.tribunnews.com/seleb/2023/01/01/agensi-mengonfirmasi-shindong-super-junior-pacaran-dengan-non-selebriti
prabowo
News does not contain the specified keywords and will not be inserted into the database. URL: https://www.tribunnews.com/pendidikan/2023/01/01/kunci-jawaban-bahasa-indonesia-kelas-10-halaman-237-penggunaan-kata-ganti-pada-teks-biografi
prabowo
News does not contain the specified keywords and will not be inserted into the database. URL: https://www.tribunnews.com/sport/2023/01/01/jadwal-motogp-2023-mulai-bulan-maret-di-portugal-motogp-mandalika-bulan-oktober
prabowo
News does not contain the specified keywords and will not 

KeyboardInterrupt: 